In [1]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

X_train = pd.read_csv("dataset/train_features.csv")
y_train = pd.read_csv("dataset/train_labels.csv")
X_test = pd.read_csv("dataset/valid_features.csv")
y_test = pd.read_csv("dataset/valid_labels.csv")

cleanup_nums = {"genre": 
                {"classic pop and rock": 1, 
                 "dance and electronica": 2,
                 "folk":3,
                 "jazz and blues":4,
                 "metal":5,
                 "pop":6,
                 "punk":7,
                 "soul and reggae":8
                }}

y_train.replace(cleanup_nums, inplace=True)
y_test.replace(cleanup_nums, inplace=True)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

X_train = X_train.drop(["trackID","loudness","title","tempo", "time_signature","key","mode","duration", "tags"], 1)
X_test = X_test.drop(["trackID","loudness","title","tempo", "time_signature","key","mode","duration", "tags"], 1)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [2]:
gnb=GaussianNB()
y_train["genre"] = y_train["genre"].astype('int')
gnb.fit(X_train, y_train["genre"])
preds=gnb.predict(X_test)

# let's how our model performed
print("Classificaion report: \n")
print(classification_report(y_test["genre"], preds))
print("Confusion matrix: \n")
print(confusion_matrix(y_test["genre"], preds))

print("\n\n\n")

def evaluate(pred, true):
    CM = metrics.confusion_matrix(true, pred) # Confusion Matrix
    Acc = metrics.accuracy_score(true, pred) # Accuracy
    precf1 = metrics.precision_recall_fscore_support(true, pred) # Precision, Recall and F1-score
    return CM, Acc, precf1

CM, Acc, precf1 = evaluate(preds, y_test["genre"])

correct = 0
for i in range(len(y_test)):
    correct = correct + int(preds[i]==y_test["genre"][i])
    
print("Confusion Matrix:\n{}\naccuracy: {}\naccuracy by sklearn.metric: {}\nprecision: {}\nrecall: {}\nF1: {}\n".format(
                                                CM,
                                                correct / len(y_test), 
                                                Acc,
                                                precf1[0],
                                                precf1[1],
                                                precf1[2]))

Classificaion report: 

              precision    recall  f1-score   support

           1       0.25      0.33      0.28        55
           2       0.48      0.31      0.38        45
           3       0.27      0.41      0.33        64
           4       0.25      0.64      0.36        44
           5       0.81      0.71      0.76        66
           6       0.48      0.18      0.26        74
           7       0.47      0.32      0.38        44
           8       0.78      0.36      0.49        58

    accuracy                           0.40       450
   macro avg       0.47      0.41      0.40       450
weighted avg       0.49      0.40      0.41       450

Confusion matrix: 

[[18  0 15 20  0  1  0  1]
 [ 7 14  4  8  2  5  2  3]
 [22  0 26 10  0  3  3  0]
 [ 1  0 11 28  0  0  4  0]
 [ 0 10  1  0 47  1  7  0]
 [11  0 29 19  0 13  0  2]
 [11  1  4  3  9  2 14  0]
 [ 3  4  5 23  0  2  0 21]]




Confusion Matrix:
[[18  0 15 20  0  1  0  1]
 [ 7 14  4  8  2  5  2  3]
 [22  0 26 1